In [1]:
'''
-----------------------------
STATIONS - PESTICIDES - STATS
-----------------------------

GOAL : notebook python functions to add at root (app initialization on run.py) 
create panda objects / implement query functions / export to JSON 
for data analysis and visualization

- READ .CSV AND .XLSX FILES (DATA) AND CONVERT IT TO PANDAS DATAFRAMES
- CHANGE COORD STATIONS TO WGS_84 (LAT/LONG)
- CLEAN AND MERGE DATA
- QUERY FUNCTIONS
- EXPORT FUNCTIONS (JSON)

AUTHOR : Julien Paris
DATE   : 24/12/2016

TO DO : 
- 
'''

'\n-----------------------------\nSTATIONS - PESTICIDES - STATS\n-----------------------------\n\nGOAL : notebook python functions to add at root (app initialization on run.py) \ncreate panda objects / implement query functions / export to JSON \nfor data analysis and visualization\n\n- READ .CSV AND .XLSX FILES (DATA) AND CONVERT IT TO PANDAS DATAFRAMES\n- CHANGE COORD STATIONS TO WGS_84 (LAT/LONG)\n- CLEAN AND MERGE DATA\n- QUERY FUNCTIONS\n- EXPORT FUNCTIONS (JSON)\n\nAUTHOR : Julien Paris\nDATE   : 24/12/2016\n\nTO DO : \n- \n'

In [2]:
### import standard libraries
import os
import itertools
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

# pyproj settings to convert coordinates
from pyproj import Proj, transform
inProj  = Proj(init='epsg:2154') # proj in  : Lambert 93
outProj = Proj(init='epsg:4326') # proj out : WSG 84

In [3]:
### basic folders addresses and names
cwd = os.getcwd()

data_folder = "app/static/data"
stats_folder = "stats"

stats_path = os.path.join(cwd, data_folder, stats_folder)

print "-- cwd :", cwd
print "-- stats path : ", stats_path

for file in os.listdir(stats_path):
    if file.endswith(".csv") or file.endswith(".xlsx"):
        print "--- dataset in '/data' : ", file
        #print cwd+datas_folder+"/"+file

-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- stats path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats
--- dataset in '/data' :  ma_qp_fm_rcsrco_pesteso_2010.csv
--- dataset in '/data' :  ma_qp_fm_rcsrco_pesteso_2011.csv
--- dataset in '/data' :  ma_qp_fm_rcsrco_pesteso_2012.csv
--- dataset in '/data' :  ma_qp_fm_ttres_pesteso_2007.csv
--- dataset in '/data' :  ma_qp_fm_ttres_pesteso_2008.csv
--- dataset in '/data' :  ma_qp_fm_ttres_pesteso_2009.csv
--- dataset in '/data' :  moy_tot_quantif_2007.csv
--- dataset in '/data' :  moy_tot_quantif_2008.csv
--- dataset in '/data' :  moy_tot_quantif_2009.xlsx
--- dataset in '/data' :  moy_tot_quantif_2010.csv
--- dataset in '/data' :  moy_tot_quantif_2011.csv
--- dataset in '/data' :  moy_tot_quantif_2012.csv
--- dataset in '/data' :  pesticides.csv
--- dataset in '/data' :  stations.csv


In [5]:
### panda dataframes for every db + settings

# set encoding of .csv (keep accents)
csv_encoding = "latin-1"

#np.array = time_frame

# root strings for datas names
root_mct = "df_mct_"
root_ma  = "df_ma_"

# list of datas filenames
datas_stations   = "stations.csv"
datas_pesticides = "pesticides.csv"

datas_MCT = [
    "moy_tot_quantif_2007.csv",
    "moy_tot_quantif_2008.csv",
    "moy_tot_quantif_2009.xlsx", #### 
    "moy_tot_quantif_2010.csv",
    "moy_tot_quantif_2011.csv",
    "moy_tot_quantif_2012.csv",
]

datas_MA = [
    "ma_qp_fm_ttres_pesteso_2007.csv",
    "ma_qp_fm_ttres_pesteso_2008.csv",
    "ma_qp_fm_ttres_pesteso_2009.csv",
    "ma_qp_fm_rcsrco_pesteso_2010.csv",
    "ma_qp_fm_rcsrco_pesteso_2011.csv",
    "ma_qp_fm_rcsrco_pesteso_2012.csv",
]

all_pesticides_code = "XXXXXX"


In [6]:
# set time frame
years   = {"ANNEE" : [2007, 2008, 2009, 2010, 2011, 2012 ] }



In [7]:
### functions : cleaning operations on dataframes

def stat_file_path(filename):
    path = os.path.join(stats_path, filename)
    return path 


def checkDTypes (df) :
    # check data type
    
    for index in df.index.names :
        print "---- index : ", index

    for col in df.columns :
        #label = col.values
        dtype = df[col].dtype
        
        print "---- dtypes col : ", col, "/", dtype
        

In [8]:
def comas2points(df, list_col_names="all_col"): 
    # convert all weird "," to "." and then to float values
    
    if list_col_names == "all_col" : 
        df.loc[:, :] = df.replace(to_replace=',', value='.', regex=True)
    else : 
        df.loc[:, list_col_names ] = df.loc[:,list_col_names].replace(to_replace=',', value='.', regex=True)
    return df


def ints2floats(df, list_col_names, to="float") :
    
    if to == "float":
        df.loc[:, list_col_names] = df.loc[:, list_col_names].astype(float)
    elif to == "int" :
        df.loc[:, list_col_names] = df.loc[:, list_col_names].astype(int)        
    return df


In [9]:
def dfCleanNa(df_list): 
    # clean from NaN values if entire row is NaN
    
    df_list_clean = []
    for df in df_list :
        df_cleaned_01 = df.dropna(how="all") # on empty rows
        df_cleaned_02 = df_cleaned_01.dropna( axis=1, how="all") # on empty columns
        df_list_clean.append(df_cleaned_02)
    
    return df_list_clean


In [10]:

#------------------------------------------------------#
#------------------------------------------------------#
#------------------------------------------------------#
#   -- DATAS TO DATA FRAMES --                         #
#------------------------------------------------------#
#------------------------------------------------------#
#------------------------------------------------------#


In [11]:

########################################################
########################################################
########################################################
### -- DF_PESTICIDES --
########################################################
########################################################
########################################################


In [12]:
functions_cols= ["CODE_FONCTION","LIBELLE CODE_FONCTION"]

functions_split = {
    "A"   : "A",
    "B"   : "B",
    "BF"  : "B,F",
    "F"   : "F",
    "FA"  : "F,A",
    "FHM" : "F,H,M",
    "FN"  : "F,N",
    "H"   : "H",
    "I"   : "I",
    "IA"  : "I,A",
    "IAFH": "I,A,F,H",
    "IAM" : "I,A,M",
    "IAN" : "I,A,N",
    "IM"  : "I,M",
    "IN"  : "I,N",
    "Ireg": "I,Reg",
    "N"   : "N",
    "R"   : "R",
    "Reg" : "Reg",
    "RepO": "RepO",
    "Ro"  : "Ro",
    "HFNI": "H,F,N,I",
    "HG"  : "H,G"
}

functions_full = {
    "A"    : "Acaricide",
    "B"    : "Biocide",
    "BF"   : "Biocide, Fongicide",
    "F"    : "Fongicide",
    "FA"   : "Fongicide, Acaricide",
    "FHM"  : "Fongicide, Herbicide, Mollusticide",
    "FN"   : "Fongicide, Nématicide",
    "H"    : "Herbicide",
    "I"    : "Insecticide",
    "IA"   : "Insecticide, Acaricide",
    "IAFH" : "Insecticide, Acaricide, Fongicide, Herbicide",
    "IAM"  : "Insecticide, Acaricide, Mollusticide",
    "IAN"  : "Insecticide, Acaricide, Nématicide",
    "IM"   : "Insecticide, Mollusticide",
    "IN"   : "Insecticide, Nématicide",
    "Ireg" : "Insecticide, Régulateur de croissance",
    "N"    : "Nématicide",
    "R"    : "Rodenticide",
    "Reg"  : "Régulateur de croissance",
    "RepO" : "Répulsif",
    "Ro"   : "Rodenticide",
    "HFNI" : "Herbicide, Fongicide, Nématicide, Insecticide",
    "HG"   : "Herbicide, Graminicide"
}


functions_light = {
    "A"   : "Acaricide",
    "B"   : "Biocide",
    "F"   : "Fongicide",
    "H"   : "Herbicide",
    "I"   : "Insecticide",
    "M"   : "Mollusticide",
    "N"   : "Nématicide",
    "R"   : "Rodenticide",
    "Reg" : "Régulateur de croissance",
    #"reg" : "Régulateur de croissance",
    "RepO": "Répulsif",
    "Ro"  : "Rodenticide",
    "G"   : "Graminicide"
}

### optional
df_functions = pd.Series(functions_light, name="LIBELLE_CODE_FONCTION")
df_functions.index.name = 'CODE_FONCTION'
df_functions.reset_index()
df_functions = df_functions.to_frame()

df_functions #["A"]


,LIBELLE_CODE_FONCTION
CODE_FONCTION,
A,Acaricide
B,Biocide
F,Fongicide
G,Graminicide
H,Herbicide
I,Insecticide
M,Mollusticide
N,Nématicide
R,Rodenticide


In [13]:
# read pesticides list

df_pesticides = pd.read_csv( stat_file_path("pesticides.csv"), sep=";", encoding=csv_encoding )
df_pesticides = comas2points(df_pesticides, ["NORME_DCE"])
df_pesticides = ints2floats (df_pesticides, ["NORME_DCE"])

##### dates  : col "DATE_NA_USAGE"
df_pesticides["DATE_NA_USAGE"] = pd.to_datetime(df_pesticides["DATE_NA_USAGE"], infer_datetime_format=True, errors='coerce')

##### index 2 by function : col "CODE_FONCTION"
df_pesticides["CODE_FONCTION"].replace(functions_split, inplace=True)

# set index : CODE_PESTICIDE 
df_pesticides.set_index(["CODE_FAMILLE", "CD_PARAMETRE", "LB_PARAMETRE"], inplace=True)
#df_pesticides.set_index(["CD_PARAMETRE", "LB_PARAMETRE"], inplace=True)
df_pesticides.sort_index(inplace=True) 

#df_pesticides["CODE_FONCTION"].head(10)


In [14]:
pest_famille_list = list(df_pesticides.index.levels[0])
print "-- len pest_famille_list", len(pest_famille_list)
print pest_famille_list
print

print " -- df_pesticides.index.names    : ", df_pesticides.index.names
print " -- df_pesticides.index.values   : ", df_pesticides.index.values
print " -- df_pesticides.columns.values : ", df_pesticides.columns.values
print " -- df_pesticides.columns        : ", df_pesticides.columns
print 

checkDTypes(df_pesticides)



-- len pest_famille_list 31
[u'Ald\xe9hydes et c\xe9tones', u'Amides', u'Amines', u'Autres \xe9l\xe9ments min\xe9raux', u'Azoles', u'Benz\xe8ne et d\xe9riv\xe9s', u'COHV, solvants chlor\xe9s, fr\xe9ons', u'Carbamate', u'Carbamates', u'Carbamates et thiocarbamates', u'Chloroacetamide ', u'Chloroac\xe9tamide', u'Chloroalcanes', u'Compos\xe9s ph\xe9noliques', u'Diazines', u'Divers (organiques)', u'Fongicides', u'Hydrocarbures et indices li\xe9s', u'Inconnu', u'Indices', u'Metaux et m\xe9tallo\xefdes', u'Organochlor\xe9s', u'Organom\xe9talliques', u'Organophosphor\xe9s', u'Pyridines', u'Pyr\xe9thrino\xefdes', u'Quinazolinones', u'Triazines et m\xe9tabolites', u'Triazoles', u'Triazolopyrimidines sulfonamides', u'Ur\xe9es']

 -- df_pesticides.index.names    :  [u'CODE_FAMILLE', u'CD_PARAMETRE', u'LB_PARAMETRE']
 -- df_pesticides.index.values   :  [(nan, 6276, u'Somme pesticides analyses')
 (nan, 6824, u'N,N-Dimet-tolylsulphamid') (nan, 6856, u'Acetochlor ESA')
 ..., (u'Ur\xe9es', 9055, u'1-(

In [15]:
idx = pd.IndexSlice
#idx[:,year, cd_parametre]
df_pesticides.loc[ idx[:,1130] , : ]["CODE_FONCTION"] #.head(3)


CODE_FAMILLE  CD_PARAMETRE  LB_PARAMETRE
Carbamates    1130          Carbofuran      I,N
Name: CODE_FONCTION, dtype: object

In [16]:
df_pesticides.loc[ idx[:,1432:1474], :] #.head(10)


NOM_PARAM2  \
CODE_FAMILLE        CD_PARAMETRE LB_PARAMETRE                       
Carbamates          1463         Carbaryl                Carbaryl   
                    1474         Chlorprophame      Chlorprophame   
Divers (organiques) 1432         Pyriméthanil        Pyriméthanil   
                    1473         Chlorothalonil    Chlorothalonil   
Organochlorés       1472         Chloropicrine      Chloropicrine   
Organophosphorés    1464         Chlorfenvinphos  Chlorfenvinphos   

                                                 CODE_FONCTION STATUT  \
CODE_FAMILLE        CD_PARAMETRE LB_PARAMETRE                           
Carbamates          1463         Carbaryl                I,Reg    PNA   
                    1474         Chlorprophame             Reg     PA   
Divers (organiques) 1432         Pyriméthanil                F     PA   
                    1473         Chlorothalonil              F     PA   
Organochlorés       1472         Chloropicrine             F,N    PNA   
Organophosphorés    1464         Chlorfenvinphos             I    PNA   

                                                 METABOLITE PARENT NOM_PARENT  \
CODE_FAMILLE        CD_PARAMETRE LB_PARAMETRE                                   
Carbamates          1463         Carbaryl               NaN    NaN        NaN   
                    1474         Chlorprophame          NaN    NaN        NaN   
Divers (organiques) 1432         Pyriméthanil           NaN    NaN        NaN   
                    1473         Chlorothalonil         NaN    NaN        NaN   
Organochlorés       1472         Chloropicrine          NaN    NaN        NaN   
Organophosphorés    1464         Chlorfenvinphos        NaN    NaN        NaN   

                                                    CODE_CAS DATE_NA_USAGE  \
CODE_FAMILLE        CD_PARAMETRE LB_PARAMETRE                                
Carbamates          1463         Carbaryl            63-25-2    2008-11-20   
                    1474         Chlorprophame      101-21-3           NaT   
Divers (organiques) 1432         Pyriméthanil     53112-28-0           NaT   
                    1473         Chlorothalonil    1897-45-6           NaT   
Organochlorés       1472         Chloropicrine       76-06-2           NaT   
Organophosphorés    1464         Chlorfenvinphos    470-90-6    2007-12-31   

                                                      FORMULEB  NORME_DCE  
CODE_FAMILLE        CD_PARAMETRE LB_PARAMETRE                              
Carbamates          1463         Carbaryl            C12H11NO2        0.1  
                    1474         Chlorprophame     C10H12ClNO2        0.1  
Divers (organiques) 1432         Pyriméthanil         C12H13N3        0.1  
                    1473         Chlorothalonil        C8Cl4N2        0.1  
Organochlorés       1472         Chloropicrine         CCl3NO2        0.1  
Organophosphorés    1464         Chlorfenvinphos  C12H14Cl3O4P        0.1

In [17]:

########################################################
########################################################
########################################################
### -- DF_STATIONS -- 
########################################################
########################################################
########################################################


In [18]:

lab_stations = "INFOS"
df_stations  = pd.read_csv( stat_file_path(datas_stations), sep=";", encoding=csv_encoding , na_values=[""] )

'''
IMPORTANT : 
name column to link to carto (.shp file) : 
"CD_ME_v2" | "CD_ME_niv1_surf"

for instance : 
"DG330" in column "CD_ME_v2" | "CD_ME_niv1_surf" in df_stations
... corresponds to :
"DG330" in column "CdMasseDEa" in gdf object (geopandas from .shp file)

''' 

# add columns CD_PARAMETRE, LB_PARAMETRE
#df_stations["CD_PARAMETRE"] = 99999
#df_stations["LB_PARAMETRE"] = "all pesticides"

# get columns labels
#col_labels_stations = list(df_stations.columns.values)
#print " -- col_labels :", df_stations[0:5]

# add multilevel hierarchy on columns
#df_stations.columns = pd.MultiIndex.from_product([lab_stations, col_labels_stations, "NO_DATE"])
#df_stations.columns = pd.MultiIndex.from_product([lab_stations, col_labels_stations])

to_float = ["ALTITUDE", "PROFONDEUR_MAXI_POINT", "X_FICT_L93", "Y_FICT_L93"]

df_stations = comas2points(df_stations, to_float)
df_stations = ints2floats (df_stations, to_float)

#print "-- indices names :", df_stations.index.name


#print df_stations["Unnamed: 26"].unique()
df_stations.drop('Unnamed: 26', axis=1, inplace=True)


In [19]:
### FOR CARTO : add column for long lat in WSG84

def convertCoordinates(row):
    x1,y1 = row["X_FICT_L93"], row["Y_FICT_L93"]
    convertedCoord = transform(inProj,outProj, x1, y1)
    return list(convertedCoord)

def extractFromList(index):
    value = row[colName][index]
    print value
    return value

df_stations["COORD_WSG84"] = df_stations.apply(convertCoordinates,axis=1)
#df_stations["LAT_WSG84"]   = df_stations.apply(lambda row: extractFromList(row['COORD_WSG84'], 0), axis=1)
#df_stations["LONG_WSG84"]  = df_stations.apply(extractFromList(index=1),axis=1)

## cf : http://chrisalbon.com/python/pandas_expand_cells_containing_lists.html
# expand df.tags into its own dataframe
coord = df_stations['COORD_WSG84'].apply(pd.Series)
# rename each variable is tags
#coord = coord.rename(columns = lambda x : 'COORD_' + str(x))
coord.columns = ["LAT_WSG84","LONG_WSG84"] 
#print coord.head()

print coord.head()
print 

# copy CD_STATION column 
df_stations["CD_STATION_"] = df_stations["CD_STATION"]

# join the tags dataframe back to the original dataframe
df_stations = pd.concat( [df_stations, coord], axis=1, join="outer" )
#df_stations.head(3)


   LAT_WSG84  LONG_WSG84
0   5.452862   46.270740
1   5.781881   45.793046
2   5.772809   45.785001
3   5.788505   45.844201
4   5.074473   45.836095



In [20]:
# set indexes for stations
#df_stations.set_index( ["CD_STATION"], inplace=True) 
df_stations.set_index(["NUM_DEP", "NOM_COM",  "CD_ME_niv1_surf", "CD_ME_v2", "CD_STATION"], inplace=True) 
df_stations.sort_index(inplace=True) 

checkDTypes(df_stations)
print "-- df_stations.shape : ", df_stations.shape

df_stations.head(8)


---- index :  NUM_DEP
---- index :  NOM_COM
---- index :  CD_ME_niv1_surf
---- index :  CD_ME_v2
---- index :  CD_STATION
---- dtypes col :  NUM_COM / object
---- dtypes col :  codagence / object
---- dtypes col :  ALTITUDE / float64
---- dtypes col :  PROFONDEUR_MAXI_POINT / float64
---- dtypes col :  Unité_coord_fictifs / object
---- dtypes col :  X_FICT_L93 / float64
---- dtypes col :  Y_FICT_L93 / float64
---- dtypes col :  reseau2009 / object
---- dtypes col :  reseau2010 / object
---- dtypes col :  reseau2011 / object
---- dtypes col :  reseau2012 / object
---- dtypes col :  reseau2013 / object
---- dtypes col :  reseau2014 / object
---- dtypes col :  fi_ma_2007 / object
---- dtypes col :  fi_ma_2008 / object
---- dtypes col :  fi_ma_2009 / object
---- dtypes col :  fi_ma_2010 / object
---- dtypes col :  fi_ma_2011 / object
---- dtypes col :  fi_ma_2012 / object
---- dtypes col :  fi_ma_2013 / object
---- dtypes col :  fi_ma_2014 / object
---- dtypes col :  COORD_WSG84 / object
-

NUM_COM  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
NaN     NaN               NaN             NaN      10261X0039/F3         NaN   
                                          CG004    01688X0034/AVAL       NaN   
                                                   01688X0039/F1         NaN   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY       01004   
        AMBLEON           DG149           DG149    07007X0001/006A     01006   
        AMBRONAY          DG389           NaN      06754X0040/007A     01007   
                                          DG389    06754X0065/P2       01007   
                                                   06754X0071/P00060   01007   

                                                                     codagence  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                    
NaN     NaN               NaN             NaN      10261X0039/F3          AEAG   
                                          CG004    01688X0034/AVAL        AERM   
                                                   01688X0039/F1          AERM   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY        AERM&C   
        AMBLEON           DG149           DG149    07007X0001/006A      AERM&C   
        AMBRONAY          DG389           NaN      06754X0040/007A      AERM&C   
                                          DG389    06754X0065/P2        AERM&C   
                                                   06754X0071/P00060    AERM&C   

                                                                      ALTITUDE  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                    
NaN     NaN               NaN             NaN      10261X0039/F3          10.0   
                                          CG004    01688X0034/AVAL       235.0   
                                                   01688X0039/F1         170.0   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY         310.0   
        AMBLEON           DG149           DG149    07007X0001/006A       420.0   
        AMBRONAY          DG389           NaN      06754X0040/007A       243.0   
                                          DG389    06754X0065/P2         243.0   
                                                   06754X0071/P00060     243.0   

                                                                      PROFONDEUR_MAXI_POINT  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                                 
NaN     NaN               NaN             NaN      10261X0039/F3                       33.0   
                                          CG004    01688X0034/AVAL                      NaN   
                                                   01688X0039/F1                       80.0   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY                        NaN   
        AMBLEON           DG149           DG149    07007X0001/006A                      NaN   
        AMBRONAY          DG389           NaN      06754X0040/007A                     12.5   
                                          DG389    06754X0065/P2                       21.0   
                                                   06754X0071/P00060                    NaN   

                                                                        Unité_coord_fictifs  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                                 
NaN     NaN               NaN             NaN      10261X0039/F3                        NaN   
                                          CG004    01688X0034/AVAL                      NaN   
                                                   01688X0039/F1                        NaN   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY            01004_ _FRDG149   
        AMBLEON           DG149           DG149    07007X0001/006A          01006_ _FRDG149   
        AM

In [21]:

########################################################
########################################################
########################################################
### -- DF_MCT (moy concentrations totales)  --
########################################################
########################################################
########################################################


In [167]:
lab_MCT = "MCT"

## read datas MCT

df_mct_2007 = pd.read_csv(  stat_file_path(datas_MCT[0]), sep=";") #,  index_col=[1,0])
df_mct_2008 = pd.read_csv(  stat_file_path(datas_MCT[1]), sep=";") #,  index_col=[1,0])
df_mct_2009 = pd.read_excel(stat_file_path(datas_MCT[2]), sep=";") #,index_col=[1,0])
df_mct_2010 = pd.read_csv(  stat_file_path(datas_MCT[3]), sep=";") #,  index_col=[1,0])
df_mct_2011 = pd.read_csv(  stat_file_path(datas_MCT[4]), sep=";") #,  index_col=[1,0])
df_mct_2012 = pd.read_csv(  stat_file_path(datas_MCT[5]), sep=";") #,  index_col=[1,0])


In [168]:
#df_mct_2007.shape
print " -- df_mct_2007.index.names : ", df_mct_2007.index.names
print " -- df_mct_2007.columns     : ", df_mct_2007.columns

#df_mct_2007.head()

 -- df_mct_2007.index.names :  [None]
 -- df_mct_2007.columns     :  Index([u'ANNEE', u'CD_STATION', u'NBPREL', u'MOYPTOT', u'MAXPTOT',
       u'MINMOLRECH', u'MAXMOLRECH', u'MINMOLQ', u'MAQMOLQ'],
      dtype='object')


In [169]:
#df_mct_2008.head() 

#df_ = df_mct_2010.dropna(how="all")
#df_.loc[:, ("ANNEE")] = df_.loc[:, ("ANNEE")].astype(int)
#df_.head() 

In [170]:
### merge all MCT datas with multiIndex
# cf : http://pandas.pydata.org/pandas-docs/stable/merging.html#joining-multiple-dataframe-or-panel-objects
# cf : http://pandas.pydata.org/pandas-docs/stable/merging.html#joining-with-two-multi-indexes
# options/alternatives : .merge .join .concat .append

frames_mct = [df_mct_2007,df_mct_2008, df_mct_2009, df_mct_2010, df_mct_2011, df_mct_2012]

# clean from NaN values if entire row is NaN
frames_mct_cleaned = dfCleanNa(frames_mct)
    
df_MCT = pd.concat(frames_mct_cleaned)

# convert all year column data to integers
df_MCT = ints2floats(df_MCT, ["ANNEE"], to="int")

# convert all weird "," to "." and then to float values
df_MCT   = comas2points(df_MCT)
to_float = ['NBPREL', 'MOYPTOT', 'MAXPTOT', 'MINMOLRECH', 'MAXMOLRECH', 'MINMOLQ', 'MAQMOLQ']
df_MCT   = ints2floats(df_MCT, to_float)

# add column CD_PARAMETRE, LB_PARAMETRE
df_MCT["CD_PARAMETRE"] = all_pesticides_code
df_MCT["LB_PARAMETRE"] = "all_pesticides"


# set index hierarchy
#df_MCT.set_index(["CD_STATION", "ANNEE"], inplace=True)
df_MCT.set_index(["CD_STATION", "ANNEE", "CD_PARAMETRE", "LB_PARAMETRE"], inplace=True)

print " -- df_MCT.index.names    : ", df_MCT.index.names
print " -- df_MCT.index.values   : ", df_MCT.index.values
print " -- df_MCT.columns.values : ", df_MCT.columns.values
print " -- df_MCT.columns        : ", df_MCT.columns

df_MCT.sort_index(inplace=True) 


 -- df_MCT.index.names    :  [u'CD_STATION', u'ANNEE', u'CD_PARAMETRE', u'LB_PARAMETRE']
 -- df_MCT.index.values   :  [('00054X0169/F1', 2007, 'XXXXXX', 'all_pesticides')
 ('00057X0245/F1', 2007, 'XXXXXX', 'all_pesticides')
 ('00057X0248/F4', 2007, 'XXXXXX', 'all_pesticides') ...,
 ('11056X0123/FIGA', 2012, 'XXXXXX', 'all_pesticides')
 ('11195X0147/FITTEL', 2012, 'XXXXXX', 'all_pesticides')
 ('11221X0134/TRAVO', 2012, 'XXXXXX', 'all_pesticides')]
 -- df_MCT.columns.values :  ['NBPREL' 'MOYPTOT' 'MAXPTOT' 'MINMOLRECH' 'MAXMOLRECH' 'MINMOLQ' 'MAQMOLQ']
 -- df_MCT.columns        :  Index([u'NBPREL', u'MOYPTOT', u'MAXPTOT', u'MINMOLRECH', u'MAXMOLRECH',
       u'MINMOLQ', u'MAQMOLQ'],
      dtype='object')


In [171]:
df_MCT["MOYPTOT_YEAR"] = np.NaN

print df_MCT.shape 
checkDTypes(df_MCT)

df_MCT.head(20)

(11144, 8)
---- index :  CD_STATION
---- index :  ANNEE
---- index :  CD_PARAMETRE
---- index :  LB_PARAMETRE
---- dtypes col :  NBPREL / float64
---- dtypes col :  MOYPTOT / float64
---- dtypes col :  MAXPTOT / float64
---- dtypes col :  MINMOLRECH / float64
---- dtypes col :  MAXMOLRECH / float64
---- dtypes col :  MINMOLQ / float64
---- dtypes col :  MAQMOLQ / float64
---- dtypes col :  MOYPTOT_YEAR / float64


NBPREL  MOYPTOT  MAXPTOT  \
CD_STATION    ANNEE CD_PARAMETRE LB_PARAMETRE                               
00054X0169/F1 2007  XXXXXX       all_pesticides     4.0   0.0000     0.00   
              2012  XXXXXX       all_pesticides     1.0   0.0000     0.00   
00057X0245/F1 2007  XXXXXX       all_pesticides     2.0   0.0000     0.00   
              2008  XXXXXX       all_pesticides     1.0   0.0200     0.02   
              2010  XXXXXX       all_pesticides     1.0   0.0000     0.00   
              2012  XXXXXX       all_pesticides     1.0   0.0000     0.00   
00057X0248/F4 2007  XXXXXX       all_pesticides     2.0   0.0200     0.04   
              2008  XXXXXX       all_pesticides     1.0   0.0000     0.00   
              2009  XXXXXX       all_pesticides     1.0   0.0000     0.00   
              2010  XXXXXX       all_pesticides     1.0   0.0000     0.00   
              2012  XXXXXX       all_pesticides     1.0   0.0000     0.00   
00061X0118/F8 2007  XXXXXX       all_pesticides     4.0   0.0125     0.02   
              2009  XXXXXX       all_pesticides     2.0   0.0250     0.04   
              2010  XXXXXX       all_pesticides     2.0   0.0450     0.06   
              2011  XXXXXX       all_pesticides     2.0   0.0150     0.03   
              2012  XXXXXX       all_pesticides     2.0   0.0100     0.02   
00066X0042/SO 2007  XXXXXX       all_pesticides     2.0   0.2800     0.35   
              2008  XXXXXX       all_pesticides     2.0   0.2700     0.45   
              2009  XXXXXX       all_pesticides     2.0   0.2900     0.35   
              2010  XXXXXX       all_pesticides     2.0   0.2950     0.30   

                                                 MINMOLRECH  MAXMOLRECH  \
CD_STATION    ANNEE CD_PARAMETRE LB_PARAMETRE                             
00054X0169/F1 2007  XXXXXX       all_pesticides        18.0        96.0   
              2012  XXXXXX       all_pesticides        64.0        64.0   
00057X0245/F1 2007  XXXXXX       all_pesticides        18.0        96.0   
              2008  XXXXXX       all_pesticides        60.0        60.0   
              2010  XXXXXX       all_pesticides        63.0        63.0   
              2012  XXXXXX       all_pesticides        64.0        64.0   
00057X0248/F4 2007  XXXXXX       all_pesticides        61.0        96.0   
              2008  XXXXXX       all_pesticides        60.0        60.0   
              2009  XXXXXX       all_pesticides        62.0        62.0   
              2010  XXXXXX       all_pesticides        63.0        63.0   
              2012  XXXXXX       all_pesticides        64.0        64.0   
00061X0118/F8 2007  XXXXXX       all_pesticides        18.0        96.0   
              2009  XXXXXX       all_pesticides        62.0        63.0   
              2010  XXXXXX       all_pesticides        63.0        63.0   
              2011  XXXXXX       all_pesticides        63.0        64.0   
              2012  XXXXXX       all_pesticides        64.0        64.0   
00066X0042/SO 2007  XXXXXX       all_pesticides        19.0        19.0   
              2008  XXXXXX       all_pesticides        31.0        31.0   
              2009  XXXXXX       all_pesticides        31.0        31.0   
              2010  XXXXXX       all_pesticides        31.0        31.0   

                                                 MINMOLQ  MAQMOLQ  \
CD_STATION    ANNEE CD_PARAMETRE LB_PARAMETRE                       
00054X0169/F1 2007  XXXXXX       all_pesticides      0.0      0.0   
              2012  XXXXXX       all_pesticides      0.0      0.0   
00057X0245/F1 2007  XXXXXX       all_pesticides      0.0      0.0   
              2008  XXXXXX       all_pesticides      1.0      1.0   
              2010  XXXXXX       all_pesticides      0.0      0.0   
              2012  XXXXXX       all_pesticides      0.0      0.0   
00057X0248/F4 2007  XXXXXX       all_pesticides      0.0      1.0   
              2008  XXXXXX       all_pesticides      0.0      0.0   
             

In [27]:

########################################################
########################################################
########################################################
### -- DF_MA (moy analyses)  --
########################################################
########################################################
########################################################


In [28]:
lab_MA = "MA"

def multilevel_MA (df, year):

    # set indexes : STATION and CODE_PESTICIDE
    #df.set_index(["CD_STATION", "CD_PARAMETRE", "LB_PARAMETRE"], inplace=True)
    #df.set_index(["CD_STATION"], inplace=True)

    # get columns labels for df_ma
    #col_labels_df_ma = list(df.columns.values)
    #print " -- col_labels :", col_labels_df_ma

    # add multilevel hierarchy on columns
    #df.columns = pd.MultiIndex.from_product([lab_MA, col_labels_df_ma, year])
    
    # convert all weird "," to "." and then to float values
    to_float = ["MA_MOY", "NORME_DCE"]
    df       = comas2points(df, to_float)
    df       = ints2floats (df, to_float)

    df["ANNEE"] = year
    
    return df


In [29]:
df_ma_2007 = pd.read_csv( stat_file_path(datas_MA[0]), sep=";", encoding = csv_encoding )
df_ma_2007 = multilevel_MA(df_ma_2007, 2007)
    
df_ma_2008 = pd.read_csv( stat_file_path(datas_MA[1]), sep=";", encoding = csv_encoding)
df_ma_2008 = multilevel_MA(df_ma_2008, 2008)

df_ma_2009 = pd.read_csv( stat_file_path(datas_MA[2]), sep=";", encoding = csv_encoding)
df_ma_2009 = multilevel_MA(df_ma_2009, 2009)

df_ma_2010 = pd.read_csv( stat_file_path(datas_MA[3]), sep=";", encoding = csv_encoding)
df_ma_2010 = multilevel_MA(df_ma_2010, 2010)

df_ma_2011 = pd.read_csv( stat_file_path(datas_MA[4]), sep=";", encoding = csv_encoding)
df_ma_2011 = multilevel_MA(df_ma_2011, 2011)

df_ma_2012 = pd.read_csv( stat_file_path(datas_MA[5]), sep=";", encoding = csv_encoding)
df_ma_2012 = multilevel_MA(df_ma_2012, 2012)


In [30]:
#df_ma_2010.head() 

#df_ma_2011.head() 

#df_ma_2012.head() 

In [31]:
### merge all MA datas 

frames_MA = [df_ma_2007, df_ma_2008, df_ma_2009, df_ma_2010, df_ma_2011, df_ma_2012]

# clean from NaN values if entire row is NaN
frames_MA_cleaned = dfCleanNa(frames_MA)

# concatenate datas MA
df_MA = pd.concat(frames_MA_cleaned)

# set index hierarchy
#df_MA.set_index(["CD_STATION"], inplace=True)
#df_MA.set_index(["CD_STATION", "ANNEE"], inplace=True)
df_MA.set_index(["CD_STATION", "ANNEE", "CD_PARAMETRE", "LB_PARAMETRE"], inplace=True)

#df_MA.sort_index(inplace=True) 
df_MA.sortlevel(inplace=True) 

print " -- df_MA.index.names    : ", df_MA.index.names
print " -- df_MA.index.values   : ", df_MA.index.values
print " -- df_MA.columns.values : ", df_MA.columns.values
print " -- df_MA.columns        : ", df_MA.columns


 -- df_MA.index.names    :  [u'CD_STATION', u'ANNEE', u'CD_PARAMETRE', u'LB_PARAMETRE']
 -- df_MA.index.values   :  [(u'00053X0002/SO1', 2007, 1102, u'Aldicarbe')
 (u'00053X0002/SO1', 2007, 1107, u'Atrazine')
 (u'00053X0002/SO1', 2007, 1108, u'Atrazine d\xe9s\xe9thyl') ...,
 (u'11282X0005/ARAGUI', 2007, 2924, u'Benfuracarbe')
 (u'11282X0005/ARAGUI', 2007, 2951, u'Iprovalicarb')
 (u'11282X0005/ARAGUI', 2007, 5475, u'Thiofanox sulfoxyde')]
 -- df_MA.columns.values :  [u'NBANASPERTS1' u'MA_MOY' u'NBQUANTIF' u'NORME_DCE']
 -- df_MA.columns        :  Index([u'NBANASPERTS1', u'MA_MOY', u'NBQUANTIF', u'NORME_DCE'], dtype='object')


In [32]:
### MA : add columns for averages and custom indicators
df_MA["MA_MOY_YEAR"] = np.NaN


In [33]:
checkDTypes(df_MA)

df_MA.head(25)


---- index :  CD_STATION
---- index :  ANNEE
---- index :  CD_PARAMETRE
---- index :  LB_PARAMETRE
---- dtypes col :  NBANASPERTS1 / int64
---- dtypes col :  MA_MOY / float64
---- dtypes col :  NBQUANTIF / int64
---- dtypes col :  NORME_DCE / float64
---- dtypes col :  MA_MOY_YEAR / float64


NBANASPERTS1  \
CD_STATION     ANNEE CD_PARAMETRE LB_PARAMETRE                            
00053X0002/SO1 2007  1102         Aldicarbe                           1   
                     1107         Atrazine                            1   
                     1108         Atrazine déséthyl                   1   
                     1109         Atrazine déisopropyl                1   
                     1136         Chlortoluron                        1   
                     1137         Cyanazine                           1   
                     1177         Diuron                              1   
                     1205         Ioxynil                             1   
                     1208         Isoproturon                         1   
                     1209         Linuron                             1   
                     1216         Méthabenzthiazuron                  1   
                     1221         Métolachlore                        1   
                     1253         Prochloraz                          1   
                     1263         Simazine                            1   
                     1266         Terbuméton                          1   
                     1268         Terbuthylazine                      1   
                     1359         Cyprodinil                          1   
                     1414         Propyzamide                         1   
                     1515         Métobromuron                        1   
                     1670         Métazachlore                        1   
                     1673         Hexazinone                          1   
                     1832         2-hydroxy atrazine                  1   
                     2045         Terbuthylazine déséthyl             1   
00053X0004/F1  2007  1102         Aldicarbe                           1   
                     1107         Atrazine                            1   

                                                           MA_MOY  NBQUANTIF  \
CD_STATION     ANNEE CD_PARAMETRE LB_PARAMETRE                                 
00053X0002/SO1 2007  1102         Aldicarbe                 0.050          0   
                     1107         Atrazine                  0.010          0   
                     1108         Atrazine déséthyl         0.005          0   
                     1109         Atrazine déisopropyl      0.005          0   
                     1136         Chlortoluron              0.010          0   
                     1137         Cyanazine                 0.010          0   
                     1177         Diuron                    0.010          0   
                     1205         Ioxynil                   0.025          0   
                     1208         Isoproturon               0.010          0   
                     1209         Linuron                   0.010          0   
                     1216         Méthabenzthiazuron        0.010          0   
                     1221         Métolachlore              0.025          0   
                     1253         Prochloraz                0.025          0   
                     1263         Simazine                  0.010          0   
                     1266         Terbuméton                0.010          0   
                     1268         Terbuthylazine            0.010          0   
                     1359         Cyprodinil                0.025          0   
                     1414         Propyzamide               0.025          0   
                     1515         Métobromuron              0.025          0   
                     1670         Métazachlore              0.025          0   
                     1673         Hexazinone                0.025          0   
                     1832         2-hydroxy atrazine        0.005          0   
                     2045         Terbuthylazine déséthyl   0.005          0   
00053X0004/F1  2007  1102        

In [34]:

########################################################
########################################################
########################################################
### --- QUERIES ON DFs
########################################################
########################################################
########################################################


In [35]:
## cf : http://pandas.pydata.org/pandas-docs/stable/indexing.html#the-query-method-experimental

def queryByIndexValue (df, indexName, indexLabelList):
    queryString = '%s in %s' %( indexLabelList, indexName)
    #print queryString
    result = df.query(queryString)
    return result
#df_sliced_01 = queryByIndexValue(df_stations, "CD_STATION", ["00066X0042/SO", "00053X0002/SO1"] )

def queryByColValue (df, colName, comparator, colValue ):
    queryString = '(%s %s %s)' %( colName, comparator, colValue)
    #print queryString
    result = df.query(queryString)
    return result


def getIndexValuesList(df, indexName):
    result = df.index.get_level_values(indexName).unique()
    return list(result)

def getColValuesList(df, colName ) :
    result = df[colName].unique()
    return list(result)


def listIndexUniqueValues(df) :
    dictIndex = {}
    for indexName in df.index.names :
        listValues = getIndexValuesList(df, indexName)
        dictIndex[indexName] = listValues
    return dictIndex


In [36]:

########################################################
########################################################
########################################################
### --- DF_AV /// by : 
###           year - pesticides (levels rows)
###           departements (levels columns) 
########################################################
########################################################



In [37]:
years_list = [2007, 2008, 2009, 2010, 2011, 2012 ] 
print "-- len years_list", len(years_list)

departements_list = list(df_stations.index.levels[0])
print "-- len departements_list", len(departements_list)
#print departements_list

pesticides_list = list(df_pesticides.index.levels[1])
pesticides_list.append(all_pesticides_code)
print "-- len pesticides_list", len(pesticides_list)
print pesticides_list

-- len years_list 6
-- len departements_list 95
-- len pesticides_list 1044
[2, 1083, 1092, 1093, 1094, 1100, 1101, 1102, 1103, 1104, 1105, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1119, 1120, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1159, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1192, 1193, 1194, 1197, 1198, 1200, 1201, 1202, 1203, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1236, 1237, 1238, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1277, 1279, 1280, 1281, 1282, 1287, 1288, 1289, 1290, 1291, 1298, 1308, 1310, 1329, 1333, 1336, 1341, 1353, 13

In [38]:
### create df_AV dataframe dummy

tuples = list(itertools.product(years_list, pesticides_list))
len_rows = len(tuples)
list_ = [np.NaN]*len_rows
dict_ = {"test" : list_ }

index = pd.MultiIndex.from_tuples(tuples, names=['year', 'CD_PARAMETRE'])
df_AV = pd.DataFrame(np.asarray(list_), index=index)
#df_AV.head()

In [39]:
for dpt in departements_list :
    df_AV[str(dpt)] = np.NaN
df_AV["TOT_FRANCE"] = np.NaN

df_AV.drop(0, axis=1, inplace=True)


In [40]:
df_AV.tail()


01  02  03  04  05  06  07  08  09  10     ...      87  88  \
year CD_PARAMETRE                                             ...               
2012 99013        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN     ...     NaN NaN   
     99020        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN     ...     NaN NaN   
     99022        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN     ...     NaN NaN   
     99024        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN     ...     NaN NaN   
     XXXXXX       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN     ...     NaN NaN   

                   89  90  91  92  93  94  95  TOT_FRANCE  
year CD_PARAMETRE                                          
2012 99013        NaN NaN NaN NaN NaN NaN NaN         NaN  
     99020        NaN NaN NaN NaN NaN NaN NaN         NaN  
     99022        NaN NaN NaN NaN NaN NaN NaN         NaN  
     99024        NaN NaN NaN NaN NaN NaN NaN         NaN  
     XXXXXX       NaN NaN NaN NaN NaN NaN NaN         NaN  

[5 rows x 96 columns]

In [41]:
### fill df_AV & correspondinf MCT | MA


In [161]:
### ---> optimization A
### pre-store staions per dpt in dict
### iterate through departements

dic_dpt_stations = {}
dic_dpt_stations_count = {}

for dpt in departements_list : 

    # get list of CD_STATION within dpt 
    stations_list = df_stations.query("NUM_DEP == '%s' " %(dpt) )
    stations_list_ = list(stations_list["CD_STATION_"])
    
    dic_dpt_stations[dpt] = stations_list_
    dic_dpt_stations_count[dpt] = len(stations_list_)
    

#print dic_dpt_stations_count

test_1 = { k: dic_dpt_stations_count[k] for k in dic_dpt_stations_count.keys()[:1]}
print test_1

test_2 = { k: dic_dpt_stations[k] for k in dic_dpt_stations.keys()[:1]}
print test_2



{u'24': 128}
{u'24': [u'08085X0023/P', u'08076X0017/ERH', u'07596X0010/F', u'07842X0005/HY', u'07821X0001/SOURCE', u'08066X0047/F', u'08066X0019/F', u'08085X0040/HY', u'07595X0022/F', u'08326X0004/HY', u'08326X0006/F', u'07584X0007/F', u'07104X0501/HY', u'08088X0015/F', u'08087X0001/HY', u'08086X0031/S', u'08085X0032/HY', u'08322X0015/P', u'07346X0002/HY', u'07345X0018/F', u'07842X0007/F2', u'08311X0001/HY', u'07583X0003/HY', u'08067X0002/HY', u'07827X0007/SOURCE', u'08305X0002/F', u'08301X0002/F', u'08305X0030/F', u'08316X0016/HY', u'07582X0005/HY', u'07827X0017/HY', u'08087X0021/F', u'07826X0010/HY', u'07107X0031/F', u'08072X0010/HY', u'07361X0014/HY', u'07361X0002/HY', u'07361X0004/S', u'07841X0019/F', u'07847X0001/HY', u'07846X0012/HY', u'07846X0013/HY', u'07348X0010/HY', u'07811X0011/F', u'08075X0014/F', u'08075X0012/HY', u'08301X0015/P', u'08065X0025/F', u'08073X0017/HY', u'08077X0030/ERH', u'08077X0005/F', u'08077X0026/S1', u'07597X0007/A25', u'07346X0013/HY', u'08066X0005/F', u

In [219]:
############ TEST ####################
### test 1 on df_MCT

print "df_MCT.shape", df_MCT.shape
print 

_years_list   = [2007, 2008]
_cd_parametre = 1177 #all_pesticides_code
_column_name  = "MA_MOY" #"MOYPTOT"
_dpt_list     = ['24', '44']

for _year in _years_list :
               
    for _dpt in _dpt_list :

        print "_year %s, _dpt %s" %(_year, _dpt)
        print "++ df_MCT.shape               ", df_MA.shape

        _df_moy_tot_year = df_MA.loc[ idx[:, _year, _cd_parametre ] , [_column_name] ]
        print "-- _df_moy_tot_year.shape    ", _df_moy_tot_year.shape

        _stations_list_temp = dic_dpt_stations[_dpt]
        #print T_stations_list_temp
        print "-- len(_stations_list_temp)  ", len(_stations_list_temp)

        _df_moy_tot_year_dpt = _df_moy_tot_year.loc[ idx[ _stations_list_temp ,:, :] , :]
        print "-- _df_moy_tot_year_dpt.shape", _df_moy_tot_year_dpt.shape

        #print "-- _df_moy_tot_year.shape    ", _df_moy_tot_year.shape
        print


df_MCT.shape (11144, 8)

_year 2007, _dpt 24
++ df_MCT.shape                (2779684, 5)
-- _df_moy_tot_year.shape     (7633, 1)
-- len(_stations_list_temp)   128
-- _df_moy_tot_year_dpt.shape (91, 1)

_year 2007, _dpt 44
++ df_MCT.shape                (2779684, 5)
-- _df_moy_tot_year.shape     (7633, 1)
-- len(_stations_list_temp)   48
-- _df_moy_tot_year_dpt.shape (45, 1)

_year 2008, _dpt 24
++ df_MCT.shape                (2779684, 5)
-- _df_moy_tot_year.shape     (4682, 1)
-- len(_stations_list_temp)   128
-- _df_moy_tot_year_dpt.shape (60, 1)

_year 2008, _dpt 44
++ df_MCT.shape                (2779684, 5)
-- _df_moy_tot_year.shape     (4682, 1)
-- len(_stations_list_temp)   48
-- _df_moy_tot_year_dpt.shape (43, 1)



In [190]:
############ TEST ####################
### test 1 on df_MCT

_station = "00057X0248/F4"
_dpt = '24'
_year = 2007
_cd_parametre = all_pesticides_code

print "station : %s / dpt : %s / year : %s" %(_station, _dpt, _year)

# get list of CD_STATION within dpt 
_stations_list = df_stations.query("NUM_DEP == '%s'" %(_dpt) )
_stations_list_ = list(_stations_list["CD_STATION_"])
print "len(_stations_list_)", len(_stations_list_)

#df_moy_dpt  = df_MCT.query('CD_STATION=="%s" and ANNEE==%s ' %(station, year) )
_df_moy_dpt_ = df_MCT.query('%s in CD_STATION and ANNEE==%s and CD_PARAMETRE == "%s" ' %( _stations_list_, _year, _cd_parametre ))

print "_df_moy_dpt_.shape", _df_moy_dpt_.shape


station : 00057X0248/F4 / dpt : 24 / year : 2007
len(_stations_list_) 128
_df_moy_dpt_.shape (36, 8)


In [187]:
############ TEST ####################
### test 2 on df_MA

_dpt = '24'
_year = 2007
_cd_parametre = str(1177)
_query_cd_parametre = "CD_PARAMETRE==%s" %(_cd_parametre)

_df_moy_ = df_MA.query("ANNEE == %s and %s" %(_year, _query_cd_parametre) )
_mean_year = _df_moy_["MA_MOY"].mean()
print pd.isnull(mean_year), ":", mean_year

#df_moy_

False : 0.0192648886816


In [225]:

###################################################################
### add columns for averages and custom indicators
def MOYPTOT_YEAR(df, year, cd_parametre, start_time):
    
    # create slicers
    idx = pd.IndexSlice

    # variables : "MOYPTOT" on df_MCT / "MA_MOY" on df_MA
    
    if cd_parametre == all_pesticides_code :
        column_name = "MOYPTOT"
        column_mean = "MOYPTOT_YEAR"
        #query_cd_parametre = 'CD_PARAMETRE == "%s"' %(cd_parametre)
        #df_moy_dpt_ = df_MCT.query('%s in CD_STATION and ANNEE==%s and CD_PARAMETRE == "%s" ' %( stations_list_, year, cd_parametre ))

    else :
        column_name = "MA_MOY"
        column_mean = "MA_MOY_YEAR"
        #query_cd_parametre = 'CD_PARAMETRE == %s' %(cd_parametre)
        #df_moy_dpt_ = df_MA.query('%s in CD_STATION and ANNEE==%s and CD_PARAMETRE == %s ' %( stations_list_, year, cd_parametre ))
    
    try : 
        #df_moy_tot_year = df.query("ANNEE == %s and %s" %(year, query_cd_parametre) )
        df_moy_tot_year = df.loc[ idx[:,year, cd_parametre] , [column_name] ]

        #print "-- %s GLOBAL - debug / mean_year %s for %s / shape df_moy_tot_year = %s" %(column_mean, year , cd_parametre, df_moy_tot_year.shape ) 

        mean_year = df_moy_tot_year[column_name].mean()
    
    except : 
        # if no cd_parametre key for this year 
        mean_year = np.NaN
    
    delta_time = datetime.now() - start_time
    print "-- %s - mean_year %s for %s : %s (delta time : %s)" %(column_mean, year , cd_parametre, mean_year, delta_time) 
    #print
    
    ### escape if mean_year == nan (leave df_Av NaN value)
    
    if pd.isnull(mean_year) == True :
        
        pass
    
    else :
        # cf : http://stackoverflow.com/questions/28002197/pandas-proper-way-to-set-values-based-on-condition-for-subset-of-multiindex-da
        # cf : http://pandas-docs.github.io/pandas-docs-travis/advanced.html#advanced-indexing-with-hierarchical-index
                
        # copy mean_year in corresponding dataframe (df)
        df.loc[ idx[:,year, cd_parametre] , [column_mean] ] = mean_year
        
        # copy mean_year in df_AV
        df_AV.loc[ idx[year, cd_parametre] , ['TOT_FRANCE'] ] = mean_year
        
        
        ### iterate through departements
        #for dpt in departements_list : 
        # optimization A -->
        for dpt, stations_list in dic_dpt_stations.iteritems() : 
                        
            # get list of CD_STATION within dpt 
            #stations_list = df_stations.query("NUM_DEP == '%s' " %(dpt) )
            #stations_list_ = list(stations_list["CD_STATION_"])
            # optimization A -->

            # compute mean for dpt
            #df_moy_dpt_year = df.query('%s in CD_STATION and ANNEE==%s and %s' %(stations_list_,year, query_cd_parametre ))
            # optimization B -->
            
            #df_moy_dpt_year = df_moy_tot_year.query('%s in CD_STATION' %(stations_list_ ))
            df_moy_tot_dpt_year = df_moy_tot_year.loc[ idx[stations_list,:, :], : ]
            
            #print "-- %s DPT - mean_year_dpt %s for %s - dpt %s (%s stations) / shape df_moy_dpt_year = %s" %(column_mean, year, cd_parametre, dpt, len(stations_list), df_moy_tot_dpt_year.shape ) 
            
            mean_year_dpt = df_moy_tot_dpt_year[column_name].mean()
            
            #print "-- %s ----- mean_year_dpt %s for %s - dpt %s (%s stations) : %s" %(column_mean, year, cd_parametre, dpt, len(stations_list), mean_year_dpt) 

            # copy mean_year_dpt in df_AV
            df_AV.loc[ idx[year, cd_parametre] , [dpt] ] = mean_year_dpt
            
            #break
            


In [226]:
### compute for df_MCT - iterate through years and dpt 

### check time deltas for eficiency 
start_time = datetime.now()

for year in years_list :

    start_lap = datetime.now()
    
    print ">>>>>>>> MOYPTOT_YEAR for %s >>>>>>>>" %(year)
    MOYPTOT_YEAR(df_MCT, year, all_pesticides_code, start_time)
    
    delta_lap = datetime.now() - start_lap
    print ">>>>>>>> finished MOYPTOT_YEAR for %s --- delta_lap : %s >>>>>>>>" %(year, delta_lap)
    print

print 
delta_time = datetime.now() - start_time
print ">>>>>>>> AVERAGE MCT --- FINISHED --- delta_time : %s" %(delta_time) 
   

>>>>>>>> MOYPTOT_YEAR for 2007 >>>>>>>>
-- MOYPTOT_YEAR - mean_year 2007 for XXXXXX : 0.0949310902745 (delta time : 0:00:00.022611)
>>>>>>>> finished MOYPTOT_YEAR for 2007 --- delta_lap : 0:00:00.945711 >>>>>>>>

>>>>>>>> MOYPTOT_YEAR for 2008 >>>>>>>>
-- MOYPTOT_YEAR - mean_year 2008 for XXXXXX : 0.119667650341 (delta time : 0:00:00.951388)
>>>>>>>> finished MOYPTOT_YEAR for 2008 --- delta_lap : 0:00:00.839522 >>>>>>>>

>>>>>>>> MOYPTOT_YEAR for 2009 >>>>>>>>
-- MOYPTOT_YEAR - mean_year 2009 for XXXXXX : 0.27284171088 (delta time : 0:00:01.792282)
>>>>>>>> finished MOYPTOT_YEAR for 2009 --- delta_lap : 0:00:00.839146 >>>>>>>>

>>>>>>>> MOYPTOT_YEAR for 2010 >>>>>>>>
-- MOYPTOT_YEAR - mean_year 2010 for XXXXXX : 0.185893663721 (delta time : 0:00:02.630894)
>>>>>>>> finished MOYPTOT_YEAR for 2010 --- delta_lap : 0:00:00.788178 >>>>>>>>

>>>>>>>> MOYPTOT_YEAR for 2011 >>>>>>>>
-- MOYPTOT_YEAR - mean_year 2011 for XXXXXX : 0.168051126112 (delta time : 0:00:03.419084)
>>>>>>>> finished MOY

In [ ]:
### WARNING : TAKES ++ TIME TO PROCESS !!!
### compute for df_MA - - iterate through years and dpt

### check time deltas for eficiency 
start_time = datetime.now()

for year in years_list :

    for pesticide in pesticides_list[:-1] :
        
        start_lap = datetime.now()
        
        print ">>>>>>>> MA_MOY_YEAR for year %s / pesticide %s " %( year, pesticide)
        MOYPTOT_YEAR(df_MA, year, pesticide, start_time )    
        
        delta_lap = datetime.now() - start_lap
        print ">>>>>>>> finished MA_MOY_YEAR for %s --- delta_lap : %s >>>>>>>>" %(year, delta_lap)
        print

print
delta_time = datetime.now() - start_time
print ">>>>>>>> AVERAGE MA --- FINISHED --- delta_time : %s" %(delta_time) 


>>>>>>>> MA_MOY_YEAR for year 2007 / pesticide 2 
-- MA_MOY_YEAR - mean_year 2007 for 2 : nan (delta time : 0:00:00.173149)
>>>>>>>> finished MA_MOY_YEAR for 2007 --- delta_lap : 0:00:00.173453 >>>>>>>>

>>>>>>>> MA_MOY_YEAR for year 2007 / pesticide 1083 
-- MA_MOY_YEAR - mean_year 2007 for 1083 : 0.0152773567083 (delta time : 0:00:00.480822)
>>>>>>>> finished MA_MOY_YEAR for 2007 --- delta_lap : 0:00:01.987650 >>>>>>>>

>>>>>>>> MA_MOY_YEAR for year 2007 / pesticide 1092 
-- MA_MOY_YEAR - mean_year 2007 for 1092 : 0.0306592423205 (delta time : 0:00:02.260392)
>>>>>>>> finished MA_MOY_YEAR for 2007 --- delta_lap : 0:00:01.603080 >>>>>>>>

>>>>>>>> MA_MOY_YEAR for year 2007 / pesticide 1093 
-- MA_MOY_YEAR - mean_year 2007 for 1093 : nan (delta time : 0:00:03.813079)
>>>>>>>> finished MA_MOY_YEAR for 2007 --- delta_lap : 0:00:00.048593 >>>>>>>>

>>>>>>>> MA_MOY_YEAR for year 2007 / pesticide 1094 
-- MA_MOY_YEAR - mean_year 2007 for 1094 : 0.0139584851583 (delta time : 0:00:03.909383)


In [221]:
df_AV.tail()

01        02        03        04  05        06  \
year CD_PARAMETRE                                                         
2012 99013              NaN       NaN       NaN       NaN NaN       NaN   
     99020              NaN       NaN       NaN       NaN NaN       NaN   
     99022              NaN       NaN       NaN       NaN NaN       NaN   
     99024              NaN       NaN       NaN       NaN NaN       NaN   
     XXXXXX        0.466632  0.159532  0.103263  0.708188 NaN  0.022414   

                         07        08     09        10     ...          87  \
year CD_PARAMETRE                                          ...               
2012 99013              NaN       NaN    NaN       NaN     ...         NaN   
     99020              NaN       NaN    NaN       NaN     ...         NaN   
     99022              NaN       NaN    NaN       NaN     ...         NaN   
     99024              NaN       NaN    NaN       NaN     ...         NaN   
     XXXXXX        0.068667  0.135451  0.362  0.143224     ...      0.0133   

                         88        89       90        91        92  93  \
year CD_PARAMETRE                                                        
2012 99013              NaN       NaN      NaN       NaN       NaN NaN   
     99020              NaN       NaN      NaN       NaN       NaN NaN   
     99022              NaN       NaN      NaN       NaN       NaN NaN   
     99024              NaN       NaN      NaN       NaN       NaN NaN   
     XXXXXX        0.048241  0.258008  0.05766  0.264667  0.096042 NaN   

                        94        95  TOT_FRANCE  
year CD_PARAMETRE                                 
2012 99013             NaN       NaN         NaN  
     99020             NaN       NaN         NaN  
     99022             NaN       NaN         NaN  
     99024             NaN       NaN         NaN  
     XXXXXX        0.76975  0.148939    0.189392  

[5 rows x 96 columns]

In [216]:
df_MCT.loc[ idx[:,2012,:,:], : ].tail()

,,,,NBPREL,MOYPTOT,MAXPTOT,MINMOLRECH,MAXMOLRECH,MINMOLQ,MAQMOLQ,MOYPTOT_YEAR
CD_STATION,ANNEE,CD_PARAMETRE,LB_PARAMETRE,,,,,,,,
10982X0003/SEGRE,2012,XXXXXX,all_pesticides,4.0,0.0000,0.000,409.0,409.0,0.0,0.0,0.189392
11013X0002/F,2012,XXXXXX,all_pesticides,5.0,0.0378,0.059,105.0,409.0,0.0,2.0,0.189392
11056X0123/FIGA,2012,XXXXXX,all_pesticides,1.0,0.0000,0.000,23.0,23.0,0.0,0.0,0.189392
11195X0147/FITTEL,2012,XXXXXX,all_pesticides,1.0,0.0000,0.000,23.0,23.0,0.0,0.0,0.189392
11221X0134/TRAVO,2012,XXXXXX,all_pesticides,1.0,0.0000,0.000,23.0,23.0,0.0,0.0,0.189392


In [38]:
df_MA.head()

NBANASPERTS1  MA_MOY  \
CD_STATION     ANNEE CD_PARAMETRE LB_PARAMETRE                                 
00053X0002/SO1 2007  1102         Aldicarbe                        1   0.050   
                     1107         Atrazine                         1   0.010   
                     1108         Atrazine déséthyl                1   0.005   
                     1109         Atrazine déisopropyl             1   0.005   
                     1136         Chlortoluron                     1   0.010   

                                                        NBQUANTIF  NORME_DCE  \
CD_STATION     ANNEE CD_PARAMETRE LB_PARAMETRE                                 
00053X0002/SO1 2007  1102         Aldicarbe                     0        0.1   
                     1107         Atrazine                      0        0.1   
                     1108         Atrazine déséthyl             0        0.1   
                     1109         Atrazine déisopropyl          0        0.1   
                     1136         Chlortoluron                  0        0.1   

                                                        MOYPTOT_YEAR  
CD_STATION     ANNEE CD_PARAMETRE LB_PARAMETRE                        
00053X0002/SO1 2007  1102         Aldicarbe                      0.0  
                     1107         Atrazine                       0.0  
                     1108         Atrazine déséthyl              0.0  
                     1109         Atrazine déisopropyl           0.0  
                     1136         Chlortoluron                   0.0

In [ ]:

########################################################
########################################################
########################################################
### tests queries 
########################################################
########################################################
########################################################


In [39]:
'''main complete and clean DF :
    - df_pesticides
    - df_stations
    - df_MCT
    - df_MA
'''

df_sliced_01 = queryByIndexValue(df_stations, "CD_STATION", ["00066X0042/SO", "00053X0002/SO1"] )
df_sliced_02 = queryByIndexValue(df_stations, "NUM_DEP", ["44"] )
df_sliced_03 = queryByIndexValue(df_MCT, "ANNEE", [2009,2010] )
df_sliced_04 = queryByIndexValue(df_MA, "CD_STATION", ["00066X0042/SO", "00053X0002/SO1"] )

df_sliced_05 = queryByColValue(df_MA, "MA_MOY", ">", "NORME_DCE")

print "-- listIndexUniqueValues : ", listIndexUniqueValues(df_sliced_02)
print
print "-- getIndexValuesList : ", getIndexValuesList(df_sliced_04, "CD_PARAMETRE") 
print
print "-- getColValuesList : ", getColValuesList(df_MA, "NORME_DCE") 
print 
#print "-- getColValuesList : ", getColValuesList(df_sliced_02, "NOM_COM")

-- listIndexUniqueValues :  {'CD_ME_niv1_surf': [u'GG117', u'GG114', u'GG038', u'GG118', u'GG026', u'GG115', u'GG022', u'GG027', u'GG139', u'GG119', u'GG140', u'GG015'], 'CD_STATION': [u'05073X0019/S9', u'04818X0181/F2', u'04818X0547/P44', u'04818X0574/F47', u'04503X0067/PZ1', u'04503X0068/PZ2', u'04503X0009/FS9', u'04503X0013/FS14', u'04503X0014/F', u'04503X0079/F9BIS', u'04507X0006/FS15', u'04507X0051/F15B', u'04502X0045/SGB2', u'04807X0018/P', u'04807X0020/F', u'04807X0048/N17', u'04503X0048/FS6', u'04502X0032/F1', u'04502X0033/F2', u'04816X0400/F', u'05353X0015/F', u'05078X0003/P3', u'05078X0033/P7', u'04193X0020/P1-1', u'04193X0025/PD2', u'04494X0014/P', u'05092X0025/PS16', u'04518X0037/NOR26', u'04518X0066/P', u'04518X0072/PB10BI', u'04514X0006/F1', u'04514X0007/F2', u'04514X0013/PB8', u'04514X0016/PB11', u'04503X0047/FS5', u'04522X0014/S', u'04513X0007/F1', u'04513X0029/F2', u'04502X0039/F', u'04498X0018/F', u'04518X0045/MSM1', u'04191X0010/P', u'05086X0028/SEL3', u'04217X0003/F

In [97]:
df_sliced_01 

NUM_COM codagence  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                         
62      CLERQUES AG001           AG001    00066X0042/SO    62228      AEAP   
        WISSANT  AG001           NaN      00053X0002/SO1   62899      AEAP   

                                                          ALTITUDE  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                 
62      CLERQUES AG001           AG001    00066X0042/SO       55.0   
        WISSANT  AG001           NaN      00053X0002/SO1      20.0   

                                                          PROFONDEUR_MAXI_POINT  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                              
62      CLERQUES AG001           AG001    00066X0042/SO                     NaN   
        WISSANT  AG001           NaN      00053X0002/SO1                    NaN   

                                                         Unité_coord_fictifs  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                           
62      CLERQUES AG001           AG001    00066X0042/SO      62228_ _FRAG001   
        WISSANT  AG001           NaN      00053X0002/SO1     62899_ _FRAG001   

                                                           X_FICT_L93  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                    
62      CLERQUES AG001           AG001    00066X0042/SO   629257.6535   
        WISSANT  AG001           NaN      00053X0002/SO1  607696.7319   

                                                           Y_FICT_L93  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                    
62      CLERQUES AG001           AG001    00066X0042/SO   7077908.332   
        WISSANT  AG001           NaN      00053X0002/SO1  7090397.222   

                                                               reseau2009  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                        
62      CLERQUES AG001           AG001    00066X0042/SO               RCS   
        WISSANT  AG001           NaN      00053X0002/SO1  Hors RCS et RCO   

                                                         reseau2010  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
62      CLERQUES AG001           AG001    00066X0042/SO     RCSseul   
        WISSANT  AG001           NaN      00053X0002/SO1        NaN   

                                                         reseau2011  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
62      CLERQUES AG001           AG001    00066X0042/SO     RCSseul   
        WISSANT  AG001           NaN      00053X0002/SO1        NaN   

                                                            ...      \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION        ...       
62      CLERQUES AG001           AG001    00066X0042/SO     ...       
        WISSANT  AG001           NaN      00053X0002/SO1    ...       

                                                         fi_ma_2008  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
62      CLERQUES AG001           AG001    00066X0042/SO         oui   
        WISSANT  AG001           NaN      00053X0002/SO1        NaN   

                                                         fi_ma_2009  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
62      CLERQUES AG001           AG001    00066X0042/SO         oui   
        WISSANT  AG001           NaN      00053X0002/SO1        NaN   

                                                         fi_ma_2010  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
62      CLERQUES AG001           AG001    00066X0042/SO         oui   
        WISSANT  AG001           NaN      00053X0002/SO1        NaN   

                                                         fi_ma_2011  \
NUM_DEP NOM_COM  CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
62      CLERQUES AG001           AG001    00066X0042/SO       

In [102]:
df_sliced_02.head(7)

NUM_COM  \
NUM_DEP NOM_COM        CD_ME_niv1_surf CD_ME_v2 CD_STATION               
44      ARTHON-EN-RETZ GG117           GG117    05073X0019/S9    44005   
        BASSE-GOULAINE GG114           GG114    04818X0181/F2    44009   
                                                04818X0547/P44   44009   
                                                04818X0574/F47   44009   
        CAMPBON        GG038           NaN      04503X0067/PZ1   44025   
                                                04503X0068/PZ2   44025   
                                       GG038    04503X0009/FS9   44025   

                                                               codagence  \
NUM_DEP NOM_COM        CD_ME_niv1_surf CD_ME_v2 CD_STATION                 
44      ARTHON-EN-RETZ GG117           GG117    05073X0019/S9       AELB   
        BASSE-GOULAINE GG114           GG114    04818X0181/F2       AELB   
                                                04818X0547/P44      AELB   
                                                04818X0574/F47      AELB   
        CAMPBON        GG038           NaN      04503X0067/PZ1      AELB   
                                                04503X0068/PZ2      AELB   
                                       GG038    04503X0009/FS9      AELB   

                                                                ALTITUDE  \
NUM_DEP NOM_COM        CD_ME_niv1_surf CD_ME_v2 CD_STATION                 
44      ARTHON-EN-RETZ GG117           GG117    05073X0019/S9       9.00   
        BASSE-GOULAINE GG114           GG114    04818X0181/F2       5.00   
                                                04818X0547/P44      3.34   
                                                04818X0574/F47      4.00   
        CAMPBON        GG038           NaN      04503X0067/PZ1      5.00   
                                                04503X0068/PZ2      5.00   
                                       GG038    04503X0009/FS9      5.31   

                                                                PROFONDEUR_MAXI_POINT  \
NUM_DEP NOM_COM        CD_ME_niv1_surf CD_ME_v2 CD_STATION                              
44      ARTHON-EN-RETZ GG117           GG117    05073X0019/S9                    13.4   
        BASSE-GOULAINE GG114           GG114    04818X0181/F2                    27.6   
                                                04818X0547/P44                   23.6   
                                                04818X0574/F47                    NaN   
        CAMPBON        GG038           NaN      04503X0067/PZ1                   38.0   
                                                04503X0068/PZ2                    8.0   
                                       GG038    04503X0009/FS9                   65.7   

                                                                  Unité_coord_fictifs  \
NUM_DEP NOM_COM        CD_ME_niv1_surf CD_ME_v2 CD_STATION                              
44      ARTHON-EN-RETZ GG117           GG117    05073X0019/S9   44005_FRGG022_FRGG117   
        BASSE-GOULAINE GG114           GG114    04818X0181/F2   44009_FRGG022_FRGG114   
                                                04818X0547/P44  44009_FRGG022_FRGG114   
                                                04818X0574/F47  44009_FRGG022_FRGG114   
        CAMPBON        GG038           NaN      04503X0067/PZ1  44025_FRGG022_FRGG038   
                                                04503X0068/PZ2  44025_FRGG022_FRGG038   
                                       GG038    04503X0009/FS9  44025_FRGG022_FRGG038   

                                                                 X_FICT_L93  \
NUM_DEP NOM_COM        CD_ME_niv1_surf CD_ME_v2 CD_STATION                    
44      ARTHON-EN-RETZ GG117           GG117    05073X0019/S9   326378.1608   
        BASSE-GOULAINE GG114           GG114    04818X0181/F2   361884.5177   
                                                04818X0547/P44  362546.7259   
                           

In [99]:
df_sliced_03.head()

NBPREL  MOYPTOT  MAXPTOT  \
CD_STATION    ANNEE CD_PARAMETRE LB_PARAMETRE                               
00057X0245/F1 2010  9999         all_pesticides     1.0    0.000     0.00   
00057X0248/F4 2009  9999         all_pesticides     1.0    0.000     0.00   
              2010  9999         all_pesticides     1.0    0.000     0.00   
00061X0118/F8 2009  9999         all_pesticides     2.0    0.025     0.04   
              2010  9999         all_pesticides     2.0    0.045     0.06   

                                                 MINMOLRECH  MAXMOLRECH  \
CD_STATION    ANNEE CD_PARAMETRE LB_PARAMETRE                             
00057X0245/F1 2010  9999         all_pesticides        63.0        63.0   
00057X0248/F4 2009  9999         all_pesticides        62.0        62.0   
              2010  9999         all_pesticides        63.0        63.0   
00061X0118/F8 2009  9999         all_pesticides        62.0        63.0   
              2010  9999         all_pesticides        63.0        63.0   

                                                 MINMOLQ  MAQMOLQ MOYPTOT_YEAR  
CD_STATION    ANNEE CD_PARAMETRE LB_PARAMETRE                                   
00057X0245/F1 2010  9999         all_pesticides      0.0      0.0               
00057X0248/F4 2009  9999         all_pesticides      0.0      0.0               
              2010  9999         all_pesticides      0.0      0.0               
00061X0118/F8 2009  9999         all_pesticides      1.0      2.0               
              2010  9999         all_pesticides      2.0      2.0

In [100]:
df_sliced_04

NBANASPERTS1  \
CD_STATION     ANNEE CD_PARAMETRE LB_PARAMETRE                                
00053X0002/SO1 2007  1102         Aldicarbe                               1   
                     1107         Atrazine                                1   
                     1108         Atrazine déséthyl                       1   
                     1109         Atrazine déisopropyl                    1   
                     1136         Chlortoluron                            1   
                     1137         Cyanazine                               1   
                     1177         Diuron                                  1   
                     1205         Ioxynil                                 1   
                     1208         Isoproturon                             1   
                     1209         Linuron                                 1   
                     1216         Méthabenzthiazuron                      1   
                     1221         Métolachlore                            1   
                     1253         Prochloraz                              1   
                     1263         Simazine                                1   
                     1266         Terbuméton                              1   
                     1268         Terbuthylazine                          1   
                     1359         Cyprodinil                              1   
                     1414         Propyzamide                             1   
                     1515         Métobromuron                            1   
                     1670         Métazachlore                            1   
                     1673         Hexazinone                              1   
                     1832         2-hydroxy atrazine                      1   
                     2045         Terbuthylazine déséthyl                 1   
00066X0042/SO  2007  1105         Aminotriazole                           2   
                     1107         Atrazine                                2   
                     1108         Atrazine déséthyl                       2   
                     1109         Atrazine déisopropyl                    2   
                     1137         Cyanazine                               2   
                     1177         Diuron                                  2   
                     1203         Hexachlorocyclohexane gamma             2   
...                                                                     ...   
               2010  1954         Hydroxyterbuthylazine                   2   
                     2045         Terbuthylazine déséthyl                 2   
               2011  1092         Prosulfocarbe                           2   
                     1105         Aminotriazole                           2   
                     1107         Atrazine                                2   
                     1108         Atrazine déséthyl                       2   
                     1109         Atrazine déisopropyl                    2   
                     1133         Chloridazone                            2   
                     1177         Diuron                                  2   
                     1184         Ethofumésate                            2   
                     1189         Fenpropimorphe                          2   
                     1203         Hexachlorocyclohexane gamma             2   
                     1208         Isoproturon                             2   
                     1209         Linuron                                 2   
                     1215         Métamitrone                             2   
                     1221         Métolachlore                            2   
                     1234         Pendiméthaline                          2   
                     1263         Simazine                                2   
                     1266         

In [101]:
df_sliced_05.head()

NBANASPERTS1  \
CD_STATION        ANNEE CD_PARAMETRE LB_PARAMETRE                                  
00061X0120/F10    2009  1113         Bentazone                                 1   
00066X0042/SO     2007  1108         Atrazine déséthyl                         2   
                  2008  1108         Atrazine déséthyl                         2   
                  2009  1108         Atrazine déséthyl                         2   
                  2010  1108         Atrazine déséthyl                         2   
                  2011  1108         Atrazine déséthyl                         2   
00075X0158/F15    2008  1506         Glyphosate                                2   
                  2009  1506         Glyphosate                                2   
00108X0004/GC1    2007  1108         Atrazine déséthyl                         1   
                  2008  1108         Atrazine déséthyl                         1   
                  2009  1108         Atrazine déséthyl                         1   
00108X0005/GC1    2007  1108         Atrazine déséthyl                         4   
                  2009  1108         Atrazine déséthyl                         2   
00111X0244/GC1    2007  1108         Atrazine déséthyl                         2   
                  2009  1108         Atrazine déséthyl                         1   
00113X0115/P2     2012  1108         Atrazine déséthyl                         1   
00115X0004/P1     2007  1108         Atrazine déséthyl                         1   
                  2008  1108         Atrazine déséthyl                         1   
00115X0030/P1     2007  1108         Atrazine déséthyl                        13   
                  2008  1108         Atrazine déséthyl                         7   
                  2009  1108         Atrazine déséthyl                         2   
                  2010  1108         Atrazine déséthyl                         2   
                  2011  1108         Atrazine déséthyl                         2   
00116X0001/P1     2007  1108         Atrazine déséthyl                         4   
                  2008  1108         Atrazine déséthyl                         2   
                  2009  1108         Atrazine déséthyl                         1   
                  2011  1108         Atrazine déséthyl                         2   
00126X0128/SO     2009  1105         Aminotriazole                             2   
                        1506         Glyphosate                                2   
                        1907         AMPA                                      2   
...                                                                          ...   
10744X0001/F      2009  1113         Bentazone                                 1   
10747X0035/HY     2007  1129         Carbendazime                              1   
10783X0006/FREDE  2009  1954         Hydroxyterbuthylazine                     1   
10795X0001/S      2008  1830         Déisopropyl-déséthyl-atrazine             3   
                  2009  1830         Déisopropyl-déséthyl-atrazine             4   
10883X0210/FAGE   2007  1177         Diuron                                    1   
10902X0002/S      2008  1830         Déisopropyl-déséthyl-atrazine             3   
                  2009  1830         Déisopropyl-déséthyl-atrazine             3   
                  2010  1830         Déisopropyl-déséthyl-atrazine             4   
                  2012  1830         Déisopropyl-déséthyl-atrazine             4   
10903X0026/PENE   2008  1830         Déisopropyl-déséthyl-atrazine             3   
                  2009  1830         Déisopropyl-déséthyl-atrazine             4   
                  2010  1830         Déisopropyl-déséthyl-atrazine             4   
                  2011  1830         Déisopropyl-déséthyl-atrazine             4   
                  2012  1830         Déisopropyl-déséthyl-atrazine             4   
10905X0014/D      2007  1506        

In [168]:

########################################################
########################################################
########################################################
### -- MERGE DATAS ??? -- 
########################################################
########################################################


In [169]:
#df_stations_MCT_MA = pd.concat( [df_stations_MCT, df_MA] )
#df_stations_MCT_MA.head()


#print df_stations_MA_MCT.columns


# pivot tables
#df_mct_2008.T

In [170]:

########################################################
########################################################
########################################################
### -- analysis --
########################################################
########################################################

## selections : http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-label
### TO DO 




In [187]:

########################################################
########################################################
########################################################
### -- exports --
########################################################
########################################################


### export functions

df_stations.head()

NUM_COM  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                
NaN     NaN               NaN             NaN      10261X0039/F3       NaN   
                                          CG004    01688X0034/AVAL     NaN   
                                                   01688X0039/F1       NaN   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY     01004   
        AMBLEON           DG149           DG149    07007X0001/006A   01006   

                                                                   codagence  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
NaN     NaN               NaN             NaN      10261X0039/F3        AEAG   
                                          CG004    01688X0034/AVAL      AERM   
                                                   01688X0039/F1        AERM   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY      AERM&C   
        AMBLEON           DG149           DG149    07007X0001/006A    AERM&C   

                                                                    ALTITUDE  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
NaN     NaN               NaN             NaN      10261X0039/F3        10.0   
                                          CG004    01688X0034/AVAL     235.0   
                                                   01688X0039/F1       170.0   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY       310.0   
        AMBLEON           DG149           DG149    07007X0001/006A     420.0   

                                                                    PROFONDEUR_MAXI_POINT  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                               
NaN     NaN               NaN             NaN      10261X0039/F3                     33.0   
                                          CG004    01688X0034/AVAL                    NaN   
                                                   01688X0039/F1                     80.0   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY                      NaN   
        AMBLEON           DG149           DG149    07007X0001/006A                    NaN   

                                                                   Unité_coord_fictifs  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                            
NaN     NaN               NaN             NaN      10261X0039/F3                   NaN   
                                          CG004    01688X0034/AVAL                 NaN   
                                                   01688X0039/F1                   NaN   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY       01004_ _FRDG149   
        AMBLEON           DG149           DG149    07007X0001/006A     01006_ _FRDG149   

                                                                     X_FICT_L93  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                     
NaN     NaN               NaN             NaN      10261X0039/F3         0.0000   
                                          CG004    01688X0034/AVAL       0.0000   
                                                   01688X0039/F1         0.0000   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY    883079.0129   
        AMBLEON           DG149           DG149    07007X0001/006A  900470.4782   

                                                                     Y_FICT_L93  \
NUM_DEP NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                     
NaN     NaN               NaN             NaN      10261X0039/F3          0.000   
                                          CG004    01688X0034/AVAL        0.000   
                                                   01688X0039/F1          0.000   
01      AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY    6544021.331   
        AMBLEON           DG149    

In [200]:
test_record = df_stations.loc[["44"], : ]
test_record 

NUM_COM  \
NUM_DEP NOM_COM                      CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
44      NaN                          NaN             NaN      10261X0039/F3         NaN   
                                                     CG004    01688X0034/AVAL       NaN   
                                                              01688X0039/F1         NaN   
        ARTHON-EN-RETZ               GG117           GG117    05073X0019/S9       44005   
        BASSE-GOULAINE               GG114           GG114    04818X0181/F2       44009   
                                                              04818X0547/P44      44009   
                                                              04818X0574/F47      44009   
        CAMPBON                      GG038           NaN      04503X0067/PZ1      44025   
                                                              04503X0068/PZ2      44025   
                                                     GG038    04503X0009/FS9      44025   
                                                              04503X0013/FS14     44025   
                                                              04503X0014/F        44025   
                                                              04503X0079/F9BIS    44025   
                                                              04507X0006/FS15     44025   
                                                              04507X0051/F15B     44025   
        DREFFEAC                     GG118           GG118    04502X0045/SGB2     44053   
        FROSSAY                      GG114           GG114    04807X0018/P        44061   
                                                              04807X0020/F        44061   
                                                              04807X0048/N17      44061   
        GUENROUET                    GG038           GG038    04503X0048/FS6      44068   
                                     GG118           GG118    04502X0032/F1       44068   
                                                              04502X0033/F2       44068   
        INDRE                        GG114           GG022    04816X0400/F        44074   
        LEGE                         GG026           GG026    05353X0015/F        44081   
        MACHECOUL                    GG117           GG117    05078X0003/P3       44087   
                                                              05078X0033/P7       44087   
        MASSERAC                     GG115           GG115    04193X0020/P1-1     44092   
                                                              04193X0025/PD2      44092   
        MISSILLAC                    GG022           GG022    04494X0014/P        44098   
        MOUZILLON                    GG027           GG027    05092X0025/PS16     44108   
        NORT-SUR-ERDRE               GG139           NaN      04518X0037/NOR26    44110   
                                                              04518X0066/P        44110   
                                                              04518X0072/PB10BI   44110   
                                                     GG139    04514X0006/F1       44110   
                                                              04514X0007/F2       44110   
                                                              04514X0013/PB8      44110   
                                                              04514X0016/PB11     44110   
        QUILLY                       GG038           GG038    04503X0047/FS5      44139   
        RIAILLE                      GG022           GG022    04522X0014/S        44144   
        SAFFRE                       GG119           GG119    04513X0007/F1       44149   
                                                              04513X0029/F2       44149   
        SAINT-GILDAS-DES-BOIS        GG118           GG118    04502X0039/F        44161   
        SAINT-MALO-DE-GUERSAC        GG022           GG022    04498X0018/F  

In [195]:

### return json 


#json_stations = df_stations.head(2).to_json(orient="split")
json_stations = df_stations.head(1).reset_index().to_json(orient="records")
print json_stations

### pretty prints
parsed = json.loads(json_stations)
print json.dumps(parsed, indent=2, sort_keys=True)

[{"NUM_DEP":null,"NOM_COM":null,"CD_ME_niv1_surf":null,"CD_ME_v2":null,"CD_STATION":"10261X0039\/F3","NUM_COM":null,"codagence":"AEAG","ALTITUDE":10.0,"PROFONDEUR_MAXI_POINT":33.0,"Unit\u00e9_coord_fictifs":null,"X_FICT_L93":0.0,"Y_FICT_L93":0.0,"reseau2009":"Hors RCS et RCO","reseau2010":null,"reseau2011":null,"reseau2012":null,"reseau2013":null,"reseau2014":null,"fi_ma_2007":"oui","fi_ma_2008":null,"fi_ma_2009":null,"fi_ma_2010":null,"fi_ma_2011":null,"fi_ma_2012":null,"fi_ma_2013":null,"fi_ma_2014":null,"COORD_WSG84":[-1.3630812101,-5.9838563092],"LAT_WSG84":-1.3630812101,"LONG_WSG84":-5.9838563092}]
[
  {
    "ALTITUDE": 10.0, 
    "CD_ME_niv1_surf": null, 
    "CD_ME_v2": null, 
    "CD_STATION": "10261X0039/F3", 
    "COORD_WSG84": [
      -1.3630812101, 
      -5.9838563092
    ], 
    "LAT_WSG84": -1.3630812101, 
    "LONG_WSG84": -5.9838563092, 
    "NOM_COM": null, 
    "NUM_COM": null, 
    "NUM_DEP": null, 
    "PROFONDEUR_MAXI_POINT": 33.0, 
    "Unit\u00e9_coord_fictifs":